This notebook is a part of motokimura's baseline solution for the [SolafuneIdentifying Deforestation Drivers competition](https://solafune.com/competitions/68ad4759-4686-4bb3-94b8-7063f755b43d?menu=about&tab=overview).
See https://github.com/motokimura/solafune_deforestation_baseline for the complete code.

> cf. @solafune (https://solafune.com) Use for any purpose other than participation in the competition or commercial use is prohibited. If you would like to use them for any of the above purposes, please contact us.

### Description

This notebook generates segmentation masks for the training images and save them as `.npy` files.
These files are used as training labels (see https://github.com/motokimura/solafune_deforestation_baseline for the training code).

Each `.npy` file contains a numpy array of shape (4, 1024, 1024) for the four classes (`grassland_shrubland`, `logging`, `mining`, and `plantation`).
The pixels with value 255 are considered to be the corresponding class.

```
data/
├── train_masks/
│   ├── train_0.npy
│   ├── train_1.npy
│   ├── train_2.npy
│   ├── ...
```

The notebook also saves the visualization of the masks along with the RGB image as a png file.
These files are just for visualization (not used for training the model).

```
data/
├── vis_train/
│   ├── train_0.png
│   ├── train_1.png
│   ├── train_2.png
│   ├── ...
```

### Requirements

#### Datasets

Download the datasets and organize them as follows:

```
data/
├── evaluation_images/
│   ├── evaluation_0.tif
│   ├── evaluation_1.tif
│   ├── evaluation_2.tif
│   ├── ...
├── train_images/
│   ├── train_0.tif
│   ├── train_1.tif
│   ├── train_2.tif
│   ├── ...
├── train_annotations.json
```

#### Libraries

Please install the python packages imported the cell below.


In [1]:
import json
from pathlib import Path

import cv2
import numpy as np
import tifffile
from tqdm import tqdm

In [2]:
data_dir = Path("./data")

In [3]:
train_file_names = [f"train_{i}.tif" for i in range(176)]  # train_0.tif ~ train_175.tif
class_names = ["grassland_shrubland", "logging", "mining", "plantation"]

with open(data_dir / "train_annotations.json", "r") as f:
    raw_annotations = json.load(f)

annotations: dict[str, dict[str, list[list[float]]]] = {}  # file_name -> class_name -> polygons
for fn in tqdm(train_file_names):
    ann: dict[str, list[list[float]]] = {}  # class_name -> polygons
    for class_name in class_names:
        ann[class_name] = []

    for tmp_img in raw_annotations["images"]:
        if tmp_img["file_name"] == fn:
            for tmp_ann in tmp_img["annotations"]:
                ann[tmp_ann["class"]].append(tmp_ann["segmentation"])

    annotations[fn] = ann

annotations["train_0.tif"]


100%|██████████| 176/176 [00:00<00:00, 106154.37it/s]


{'grassland_shrubland': [],
 'logging': [],
 'mining': [],
 'plantation': [[0.0,
   449.0,
   9.0,
   454.0,
   18.0,
   461.0,
   26.0,
   468.0,
   33.0,
   475.0,
   40.0,
   477.0,
   50.0,
   485.0,
   59.0,
   497.0,
   70.0,
   502.0,
   78.0,
   513.0,
   82.0,
   523.0,
   91.0,
   536.0,
   97.0,
   546.0,
   98.0,
   556.0,
   102.0,
   558.0,
   111.0,
   578.0,
   122.0,
   603.0,
   132.0,
   627.0,
   133.0,
   635.0,
   113.0,
   641.0,
   86.0,
   642.0,
   86.0,
   642.0,
   71.0,
   643.0,
   69.0,
   654.0,
   65.0,
   665.0,
   58.0,
   674.0,
   51.0,
   681.0,
   56.0,
   694.0,
   57.0,
   700.0,
   80.0,
   695.0,
   83.0,
   700.0,
   88.0,
   702.0,
   88.0,
   703.0,
   102.0,
   699.0,
   113.0,
   695.0,
   122.0,
   696.0,
   131.0,
   705.0,
   144.0,
   715.0,
   151.0,
   727.0,
   155.0,
   730.0,
   165.0,
   725.0,
   180.0,
   721.0,
   187.0,
   723.0,
   198.0,
   732.0,
   223.0,
   742.0,
   250.0,
   753.0,
   261.0,
   757.0,
   261.0,
   769

In [4]:
mask_save_dir = data_dir / "train_masks"
mask_save_dir.mkdir(parents=True, exist_ok=True)

for fn in tqdm(train_file_names):
    mask = np.zeros((4, 1024, 1024), dtype=np.uint8)
    anns = annotations[fn]
    for class_idx, class_name in enumerate(class_names):
        for polygon in anns[class_name]:
            cv2.fillPoly(mask[class_idx], [np.array(polygon).astype(np.int32).reshape(-1, 2)], 255)

    np.save(mask_save_dir / fn.replace(".tif", ".npy"), mask)

100%|██████████| 176/176 [00:00<00:00, 577.25it/s]


In [5]:
# visualize masks and save as a png file along with the RGB image

vis_save_dir = data_dir / "vis_train"
vis_save_dir.mkdir(parents=True, exist_ok=True)

for fn in tqdm(train_file_names):
    mask = np.load(mask_save_dir / fn.replace(".tif", ".npy"))
    vis_masks = [np.zeros((1024, 1024, 3), dtype=np.uint8) for _ in range(4)]
    for class_idx, class_name in enumerate(class_names):
        vis_masks[class_idx][mask[class_idx] > 0] = np.array([255, 0, 0])  # blue
        # put class_name as text on the mask
        cv2.putText(vis_masks[class_idx], class_name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    vis_image = tifffile.imread(data_dir / "train_images" / fn)
    vis_image = vis_image[:, :, [1, 2, 3]]  # extract BGR channels (B2, B3, and B4 band of Sentinel-2)
    vis_image = np.nan_to_num(vis_image, nan=0)
    vis_image = (vis_image / 8).clip(0, 255).astype(np.uint8)

    partition = np.ones((1024, 5, 3), dtype=np.uint8) * 255  # white partition
    vis = np.concatenate([vis_image, partition, vis_masks[0], partition, vis_masks[1], partition, vis_masks[2], partition, vis_masks[3]], axis=1)
    cv2.imwrite(vis_save_dir / fn.replace(".tif", ".png"), vis)


100%|██████████| 176/176 [00:23<00:00,  7.55it/s]
